In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
#import pyautogui
import time
from PIL import Image
import os
from pathlib import Path
from datetime import datetime, timezone

class Dota2Dataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = Path(data_dir)
        self.transform = transform
        
        # Encontrar todos os heróis e seus estados (pick/ban)
        self.classes = []
        for path in self.data_dir.glob("*_*"):
            if path.is_dir():
                self.classes.append(path.name)  # Formato: "heroname_state"
        
        self.classes = sorted(self.classes)
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        
        self.images = []
        self.labels = []
        
        # Carregar imagens e labels
        for class_name in self.classes:
            class_dir = self.data_dir / class_name
            for img_path in class_dir.glob('*.png'):
                self.images.append(str(img_path))
                self.labels.append(self.class_to_idx[class_name])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
            
        return image, label

class Dota2HeroClassifier:
    def __init__(self, model_path=None, data_dir='hero_dataset'):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        # Definir transformações para as imagens
        self.transform = transforms.Compose([
            transforms.Resize((256, 144)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        
        # Carregar dataset e criar modelo
        self.dataset = Dota2Dataset(data_dir, self.transform)
        
        self.model = self._create_model()
        
        if model_path and os.path.exists(model_path):
            self.model.load_state_dict(torch.load(model_path, map_location=self.device))
            print(f"Modelo carregado de {model_path}")
            
        self.model = self.model.to(self.device)
        
        # Criar mapeamento reverso de índice para classe
        self.idx_to_class = {v: k for k, v in self.dataset.class_to_idx.items()}
    
    def _create_model(self):
        """Cria um modelo MobileNetV2 pré-treinado e modifica a última camada"""
        model = models.mobilenet_v2(pretrained=True)
        
        # Congelar os pesos do modelo base
        for param in model.parameters():
            param.requires_grad = False
            
        # Modificar a última camada para o número de classes
        num_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_features, len(self.dataset.classes))
        
        return model
    
    def train(self, epochs=10, batch_size=16, learning_rate=0.00001):
        """Treina o modelo com o dataset fornecido"""
        self.model.train()
        
        train_loader = DataLoader(self.dataset, batch_size=batch_size, shuffle=True)
        optimizer = optim.Adam(self.model.classifier.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()
        
        for epoch in range(epochs):
            running_loss = 0.0
            correct = 0
            total = 0
            
            for images, labels in train_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            
            epoch_loss = running_loss / len(train_loader)
            accuracy = 100 * correct / total
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')
    
#     def predict_image(self, image):
#         """Prediz o herói e seu estado em uma imagem"""
#         self.model.eval()
        
#         # Converter imagem para PIL se necessário
#         if isinstance(image, np.ndarray):
#             image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
#         # Aplicar transformações
#         image_tensor = self.transform(image).unsqueeze(0).to(self.device)
        
#         with torch.no_grad():
#             outputs = self.model(image_tensor)
#             _, predicted = torch.max(outputs, 10)
#             confidence = torch.nn.functional.softmax(outputs, dim=1).max().item()
#             print(f'Confidence {confidence}')
#             if confidence > 0.8:  # Ajuste este limiar conforme necessário
#                 class_name = self.idx_to_class[predicted.item()]
#                 hero_name, state = class_name.split('_')
#                 return {'hero': hero_name, 'state': state, 'confidence': confidence}
#             return None

    def predict_image(self, image, top_k=8):
        """Prediz os heróis e seus estados em uma imagem, retornando os top_k resultados"""
        self.model.eval()

        # Converter imagem para PIL se necessário
        if isinstance(image, np.ndarray):
            image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Aplicar transformações
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)

        with torch.no_grad():
            outputs = self.model(image_tensor)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            top_probs, top_indices = torch.topk(probabilities, top_k, dim=1)

            predictions = []
            for i in range(top_k):
                confidence = top_probs[0, i].item()
                class_idx = top_indices[0, i].item()
                class_name = self.idx_to_class[class_idx]
                
                state = class_name.split('_')[-1]
                
                hero_name = class_name[:len(class_name)-len(state)-1]
                predictions.append({'hero': hero_name, 'state': state, 'confidence': confidence})

            return predictions
    
    def analyze_draft_image(self, image_path):
        """Analisa uma imagem estática da tela de draft"""
        # Carregar imagem
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Não foi possível carregar a imagem: {image_path}")
            
        height, width = image.shape[:2]
        mid_width = width // 2
        pick_height = int(height * 0.1)
        
        # Dividir a imagem em lado radiant (esquerdo) e dire (direito)
        radiant_side = image[:pick_height, :mid_width]
        dire_side = image[:pick_height, mid_width:]
        
        results = {
            'radiant': {'picks': [], 'bans': []},
            'dire': {'picks': [], 'bans': []}
        }
        
        # Analisar lado radiant
        predictions = self.process_team_side(radiant_side)
        for pred in predictions:
            if pred['state'] == 'pick':
                results['radiant']['picks'].append(pred['hero'])
            else:
                results['radiant']['bans'].append(pred['hero'])
        
        # Analisar lado dire
        predictions = self.process_team_side(dire_side)
        for pred in predictions:
            if pred['state'] == 'pick':
                results['dire']['picks'].append(pred['hero'])
            else:
                results['dire']['bans'].append(pred['hero'])
        
        return results
    
    def process_team_side(self, image):
        """Processa um lado da imagem (radiant ou dire) para encontrar heróis"""
        # Obtém o timestamp atual em UTC
        timestamp = datetime.now(timezone.utc).timestamp()
        # Salvar a imagem
        cv2.imwrite(str(timestamp) + '.png', image)
        
        predictions = []
        height, width = image.shape[:2]
        
        result = self.predict_image(image)
        print(result)
        
        return predictions
        
#         # Você precisará ajustar estes parâmetros baseado no seu layout
#         regions = []
        
#         # Exemplo de regiões para picks (ajuste conforme necessário)
#         pick_y = height // 4
#         for i in range(5):  # 5 picks por time
#             x = (width // 6) * (i + 1)
#             regions.append((x - 50, pick_y - 50, 100, 100))  # (x, y, width, height)
        
#         # Exemplo de regiões para bans (ajuste conforme necessário)
#         ban_y = height * 3 // 4
#         for i in range(7):  # 7 bans por time
#             x = (width // 8) * (i + 1)
#             regions.append((x - 50, ban_y - 50, 100, 100))
        
#         # Analisar cada região
#         for x, y, w, h in regions:
#             roi = image[y:y+h, x:x+w]
#             prediction = self.predict_image(roi)
#             if prediction:
#                 predictions.append(prediction)
        
#         return predictions
    
#     def run_live(self, interval=0.5):
#         """Executa o classificador em tempo real"""
#         try:
#             while True:
#                 screenshot = np.array(pyautogui.screenshot())
#                 results = self.analyze_draft_image(screenshot)
                
#                 print("\nStatus atual:")
#                 print("Radiant picks:", results['radiant']['picks'])
#                 print("Radiant bans:", results['radiant']['bans'])
#                 print("Dire picks:", results['dire']['picks'])
#                 print("Dire bans:", results['dire']['bans'])
                
#                 time.sleep(interval)
#         except KeyboardInterrupt:
#             print("\nMonitoramento encerrado pelo usuário.")
    
    def save_model(self, path='dota2_model.pth'):
        """Salva o modelo treinado"""
        torch.save(self.model.state_dict(), path)
        print(f"Modelo salvo em {path}")

# Função auxiliar para teste
def test_draft_image(model_path, image_path):
    """Testa o modelo com uma imagem estática de draft"""
    classifier = Dota2HeroClassifier(model_path=model_path)
    results = classifier.analyze_draft_image(image_path)
    
    print("\nResultados da análise:")
    print("\nRadiant:")
    print(f"Picks: {results['radiant']['picks']}")
    print(f"Bans: {results['radiant']['bans']}")
    print("\nDire:")
    print(f"Picks: {results['dire']['picks']}")
    print(f"Bans: {results['dire']['bans']}")
    
    return results

In [ ]:
classifier = Dota2HeroClassifier()
classifier.train(epochs=100)
classifier.save_model('modelo_1.pth')

In [ ]:
test_draft_image('dota2_model.pth', 'DOTA_2_DRAFT.png')